# Notifications and Secrets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/07_production/secrets_management/notifications_and_secrets/notifications_secrets.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/07_production/secrets_management/notifications_and_secrets/notifications_secrets.ipynb)

## Business Scenario

Production pipelines need secure secret resolution and alerting on failures or quarantines.

## Value Proposition

- Centralize secret management
- Route notifications to Slack/Teams/Email
- Keep contracts free of hard-coded secrets

---

## Goals

1. Configure secret providers
2. Set up notifications
3. Test alerts


## 1. The Strategy
LakeLogic supports multiple notification channels:
- Slack
- Teams
- SMTP (Email)
- SendGrid
- Webhooks

Critically, it supports **Secret Resolution** from:
- Environment Variables
- Azure Key Vault
- AWS Secrets Manager
- GCP Secret Manager
- HashiCorp Vault
- Encrypted Local Secrets File


## 2. Using Environment Variables
The simplest way to hide secrets is using the `env:` prefix.


In [1]:
import os
from lakelogic import DataProcessor

# Mock environment variables
os.environ["SLACK_WEBHOOK"] = "https://hooks.slack.com/services/YOUR/SECRET/URL"
os.environ["SMTP_PASSWORD"] = "super-secret-password"


### Example Contract with Env Secrets
```yaml
quarantine:
  enabled: true
  strict_notifications: false
  notifications:
    - type: slack
      target: "env:SLACK_WEBHOOK"
      on_events: ["quarantine"]
    - type: smtp
      target: "alerts@company.com"
      smtp_password: "${ENV:SMTP_PASSWORD}"
```


## 3. Cloud Secret Managers
In production, you should use a proper secret manager. LakeLogic makes this as easy as changing a string.


### Azure Key Vault
```yaml
    - type: sendgrid
      api_key: "keyvault:sendgrid-api-key"
      key_vault_url: "https://my-vault.vault.azure.net/"
```

### AWS Secrets Manager
```yaml
    - type: smtp
      smtp_password: "aws:lakelogic/prod/smtp-password"
```

### GCP Secret Manager
```yaml
    - type: webhook
      target: "gcp:target-webhook-url"
      gcp_project: "my-project-id"
```

### Encrypted Local Secrets File
```yaml
    - type: smtp
      smtp_password: "local:smtp_password"
      secrets_file: "./secrets.enc"
      secrets_key: "env:LAKELOGIC_SECRETS_KEY"
```



## 4. Run-time Resolution
When you call `processor.run()`, LakeLogic automatically resolves these strings before initializing the notification adapters. This means your YAML contracts are safe to commit to Git!

Tip: Set `strict_notifications: false` if you want a run to continue even when a notification fails.

